In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [ ]:
##load the trained model,scaler pickle ,onehot
model=load_model("model.keras")

with open("label_encoder_gender.pkl","rb")as f1:
    label_enocder_geo=pickle.load(f1)
with open("onehot_encoder_geography.pkl","rb")as f:
    one_hot_goe=pickle.load(f)
with open("scaler.pkl","rb")as f:
    scaler=pickle.load(f)

In [3]:
#example input data

input_data={
    "CreditScore":600,
    "Geography":"France",
    "Gender":"Male",
    "Age":40,
    "Tenure":3,
    "Balance":60000,
    "NumOfProducts":2,
    "HasCrCard":1,
    "IsActiveMember":1,
    "EstimatedSalary":50000
}

In [8]:
df=pd.DataFrame([input_data])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [12]:

g=one_hot_goe.transform([df["Geography"]])
g

c:\Users\swath\Desktop\data science with langchain\myenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [13]:
gen=pd.DataFrame(g,columns=one_hot_goe.get_feature_names_out())
gen

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [18]:
input_data=pd.concat([df.reset_index(drop=True),gen],axis=1)
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [19]:
gender=label_enocder_geo.transform([input_data["Gender"]])
gender

c:\Users\swath\Desktop\data science with langchain\myenv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([1])

In [20]:
input_data["Gender"]=gender
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [24]:
input_data.drop("Geography",axis=1,inplace=True)

In [25]:
scaled_data=scaler.transform(input_data)

In [27]:
scaled_data

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [29]:
prd=model.predict(scaled_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step


In [30]:
prediction_proba=prd[0][0]
prediction_proba

np.float32(0.026583768)

In [33]:
if prediction_proba>0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
